Берем лучший классификатор

best_cols=['f138', 'f96', 'f76', 'f11', 'f185', 'f115', 'f131', 'f83']

clf = RandomForestClassifier(max_features='auto',n_estimators=300,max_depth=100)

clf.fit(X[best_cols],y)

predict_result(clf,X_test[best_cols])

Добавляем в него последовательно фичи из F^2 методом adddel


In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from random import randint

In [4]:
def predict_result(clf,X_test):
    Y_pred=clf.predict(X_test)
    res_file=open('result.csv', 'w')
    for y_item in Y_pred:
      res_file.write("%s\n" % y_item)
    res_file.close()

In [5]:
X=pd.read_csv('x_train.csv',delimiter=";",header=None,names=['f'+str(i) for i in range (223)])
y=pd.read_csv('y_train.csv',names=['class'])
y=y['class']
X_test=pd.read_csv('x_test.csv',delimiter=";",header=None,names=['f'+str(i) for i in range (223)])

In [6]:
def get_cv_score(cur_set):
    if len(cur_set)==0:
        return 0
    rf=RandomForestClassifier(max_features='auto',n_estimators=300,max_depth=100,random_state=1)
    return (np.mean(cross_val_score(rf, X[cur_set], y, cv=5)))

In [7]:
# Вспомогательная фуникця для ADD шага
# Определяем признак, добавление которого в набор приводит к лучшему изменению качества
def add_one_step(ftr_set):
    cols=X.columns
    cv_df=pd.DataFrame(columns=['score'])
    # Последовательно перебирем все признаки, которые можем добавить
    print (   "currentFtr=",end=" ")
    for ftr in cols:
        #  и которые уже не входят в текущий набор признаков
        if ftr in ftr_set:
            continue
        # оцениваем качество для очередного признака
        cur_ftr_set=ftr_set.copy()
        cur_ftr_set.append(ftr)
        print (   ftr,end=" ")
        cv_df.loc[ftr]= get_cv_score(cur_ftr_set)
    # Возвращаем признак на котором достигли лучшего качества и саму метрику
    return (cv_df.idxmax()[0], cv_df.max()[0])

# Функция делает полный ADD шаг
# Возвращает количество признаков, которые добавлены с увеличение качества
def add_step(cur_set):
    print ("===Start ADD step")
    DoAdd=True
    cur_cv=get_cv_score(cur_set)
    stps=0
    # Пытаемся добавлять признаки, пока растет качество
    while DoAdd:
        # Добавляем очередной признак и считаем качество
        add_ftr,add_cv=add_one_step(cur_set)
        print ("Step=",stps," Curset=",cur_set,"  CurCV=",cur_cv)
        print ("  BestFtr=",add_ftr,"  BestCV=",add_cv)
        # Качество выросло?
        if add_cv>cur_cv:
            # Если да, то добавляем признак в итоговый набор признаков
            cur_set.append(add_ftr)
            cur_cv=add_cv
            stps=stps+1
            print ("  append ftr")
        # Если качество не растет, то заканчиваем ADD шаг
        else: DoAdd=False
    print ("END Add step. Curset=",cur_set,"  CurCV=",cur_cv) 
    return stps,cur_set

# Вспомогательная фуникця для DEL шага    
# Определяем признак, исключение которого из набор приводит к лучшему изменению качества
def del_one_step(ftr_set):
    cv_df=pd.DataFrame(columns=['score'])
    # Последовательно перебирем все признаки из исходного множества
    for ftr in ftr_set:
        # исключаем признак из набора
        cur_ftr_set=ftr_set.copy()
        cur_ftr_set.remove(ftr)
        # считаем качество
        cv_df.loc[ftr]= get_cv_score(cur_ftr_set)
    # Возвращаем признак на котором достигли лучшего качества и саму метрику
    return (cv_df.idxmax()[0], cv_df.max()[0])

# Функция делает полный DEL шаг
# Возвращает количество признаков, которые удалены с увеличение качества
def del_step(cur_set):
    print ("===Start DEL step")
    DoDel=True
    cur_cv=get_cv_score(cur_set)
    stps=0
    # Пытаемся удалять признаки, пока растет качество
    while DoDel:
        # Удаляем очередной признак и считаем качество
        del_ftr,del_cv=del_one_step(cur_set)
        print ("Step=",stps," Curset=",cur_set,"  CurCV=",cur_cv)
        print ("  BestFtr=",del_ftr,"  BestCV=",del_cv)
        # Качество выросло?
        if del_cv>cur_cv:
            # Если да, то добавляем признак в итоговый набор признаков
            cur_set.remove(del_ftr)
            cur_cv=del_cv
            stps=stps+1
            print ("  delete ftr")
        # Если качество не растет, то заканчиваем ADD шаг
        else: DoDel=False
    print ("End DEL step. Curset=",cur_set,"  CurCV=",cur_cv) 
    return stps,cur_set

In [ ]:
cur_set=[]
k=5
while k>0:
    print ("=Start ADD-DEL Step")
    add_st,cur_set=add_step(cur_set)   
    print (">",add_st,cur_set)
    del_st,cur_set=del_step(cur_set)
    print (">",del_st,cur_set)
    k=del_st+add_st
    #k=k+1

In [13]:
best_cols=['f138', 'f96', 'f76', 'f11', 'f185', 'f115', 'f131']
rf=RandomForestClassifier(max_features='auto',n_estimators=300,max_depth=100,random_state=1)
print (np.mean(cross_val_score(rf, X[best_cols], y, cv=5)))

0.630277504733


In [14]:
print (np.mean(cross_val_score(rf, X[best_cols], y, cv=3)))

0.615928323433


In [15]:
X_ftr=pd.read_csv('ftr_train.csv')

In [ ]:
rf=RandomForestClassifier(max_features='auto',n_estimators=300,max_depth=100,random_state=1)
ftr_set=['f138', 'f96', 'f76', 'f11', 'f185', 'f115', 'f131']
best_cv=np.mean(cross_val_score(rf, X_ftr[ftr_set], y, cv=5,))
print ("init cv=",best_cv)
while True:
    # Берем очередной признак
    rnd_col=randint(223,25198)
    ftr=X_ftr.columns[rnd_col]
    # формируем набор признаков с учетом нового
    cur_ftr_set=ftr_set.copy()
    cur_ftr_set.append(ftr)
    cur_cv=np.mean(cross_val_score(rf, X_ftr[cur_ftr_set], y, cv=5))
    f = open('add_del_f_square.txt', 'a')    
    if (cur_cv>best_cv):
        f.write ("!")
    f.writelines(ftr+" "+str(cur_cv)+ '\n')
    f.close()
    

init cv= 0.630277504733
